# argouml

## TABLE 4.1  Sebaran tingkat kebahayaan
pie chart nya di simpan di google spreadsheet <a href="#">argouml - Data Eksperimen</a>

### filter lanza
flow chart perlu menambahkan alir untuk filter kebahayaan berdasarkan kriteria lanza dan marinescu

### kapan refactoring berhenti ?
harm and problem class is exists ?
ya : kembali refactoring
no : stop refactoring

### Process sqlite from JCodeOdor
Jangan lupa buat view di db sqlite nya, seperti:
<ol>
    <li>detect_dcoh</li>
    
</ol>

<pre>CREATE VIEW detect_dcoh as
select 
meth.id as method_id,
type.name as type,meth.name,
value
from 
(select ms.id , ms.parent,ms.name,value 
	from Measurables as ms left join Measures as me 
on ms.id = me.measurable
where key ='DispersedCouplingHarmfulness' and value > 0
order by value desc) as meth
left join 
(select id ,name from Measurables  where type ='type' ) as type
on meth.parent = type.id
</pre>

langkah selanjutnya adalah ?
<ol>
    <li> <b>init class-harmfulnes.xlsx </b>  mencatat file berbahaya di file ini</li>
    <li> <b>init tahap-awal.xlsx </b>  mencatat file yang sudah mengalami perubahan di file ini </li>    
</ol>

In [6]:
import pandas as pd    
import numpy as np
import sqlite3 as sql

class Harmfulness:
    def __init__(self, db):
        self.conn = sql.connect('sqlite-jcodeodor/database/'+db)
    
    def specific_method(self,class_str):
#         class_name = 'WizBreakCircularComp'
#         classes = ['CrNoOutgoingTransitions','WizAssocComposite']            
#         classes_str = ', '.join(classes)
        
        sql_meth = "(select ms.id , ms.parent,ms.name,key,value from Measurables as ms left join Measures as me on ms.id = me.measurable order by value desc)"
        sql1 = "select meth.id as method_id, type.name as type,meth.name, meth.key, value from "+sql_meth+" as meth left join "        
        sql2 = "(select id ,name from Measurables  where type ='type' ) as type on meth.parent = type.id "                
        where_class = "where type.name in ('"+class_str+"') "               
        sql = sql1+sql2+where_class
        return pd.read_sql(sql,self.conn)      
            
    def dist(self): 
        return pd.read_sql('select name,type,value from detect_dcoh',self.conn)
    
    def print_excel(self): 
        dist = self.dist()
        return dist
    
    def get_table(self):    
        harm_sql = 'value = 10 '
        problem_sql = 'value >= 7.75 and value < 10'
        trouble_sql = 'value >= 5.5 and value < 7.75'
        warn_sql = 'value >= 3.25 and value < 5.5'
        bother_sql = 'value >= 1 and value < 3.25'
        harm = self.get_total(harm_sql)
        warn = self.get_total(warn_sql)
        trouble = self.get_total(trouble_sql)
        problem = self.get_total(problem_sql)
        bother = self.get_total(bother_sql)
        arr = np.array([ ['Harm', harm], ['Problem', problem], 
        ['Trouble', trouble], ['Warning', warn],
                        ['Bother', bother]])
        df2 = pd.DataFrame(arr, columns=['level', 'frekuensi'])
        return df2

    def get_total(self,level_sql):        
        base_sql = 'select count(*) as total from detect_dcoh where'    
        # disini try catch jika detect_dcoh belum dibuat
        # OperationalError: no such table: detect_dcoh
#         harus buat detect_dcoh dulu
        return pd.read_sql(base_sql +' '+ level_sql, self.conn).total[0]
    
    def is_view_has_created(self):
#         jika view belum dibuat, langsung error bahwa table detect_dcoh belum dibuat
        if(self.conn):
            print('yes, detect_dcoh has created')            
        else:
            print('gagal')
    
    def list_dch(self):
        sql = "select ms.id , ms.parent,ms.name,value from Measurables as ms left join Measures as me on ms.id = me.measurable where key ='DispersedCouplingHarmfulness' order by value desc"
        return pd.read_sql(sql,self.conn)
    
tahap0 = Harmfulness('1-tahap-0-argouml.sqlite')
tahap0.is_view_has_created()
# tahap0.list_dch()
tahap1 = Harmfulness('2-tahap-1-argouml.sqlite')
tahap1.is_view_has_created()

yes, detect_dcoh has created
yes, detect_dcoh has created


In [7]:
# kriteria smell dco menurut lanza hardly avoidable
# berdasarkan kriteria lanza, beberapa kelas yang terdeteksi berbahaya tidak ikut direfactoring.
# berikut daftar kelas nya.
# tahap0.dist().to_excel("argouml-class-harmfulnes.xlsx")
# tahap1.dist().to_excel("argouml-class-harmfulnes-tahap-2.xlsx")
# class_str = 'SelectionReshape'
# tahap0.specific_method(class_str)
# tahap0.specific_method(class_str).to_excel("argouml-"+class_str+".xlsx")
# tahap0.list_by_fanout()
# tahap0.list_by_fanout().to_excel("argouml-FANOUT.xlsx")

In [7]:
class CustomQuery(Harmfulness):    
    def get_number_of_classes(self):                
        measurable_key = 'NOCS'
        sql_meth = "(select ms.id , ms.parent, ms.name, key,value from Measurables as ms LEFT JOIN Measures as me on ms.id = me.measurable where key ='"+measurable_key+"' order by value desc)"        
        
        sql1 = "select meth.id as method_id,meth.name, meth.key, value from "+sql_meth+" as meth left join "        
        
        sql2 = "(select id ,name from Measurables  where type ='type' ) as type on meth.parent = type.id "                                
#         where_class = "where type.name in ('SelectionManager') "
#         sql = sql1+sql2+where_class
        sql = sql1+sql2
        return pd.read_sql(sql,self.conn)
    
    def list_by_cint(self):                
        measurable_key = 'CINT'
        sql_meth = "(select ms.id , ms.parent,ms.name,key,value from Measurables as ms LEFT JOIN Measures as me on ms.id = me.measurable where key ='"+measurable_key+"' order by value desc)"        
        sql1 = "select meth.id as method_id, type.name as type,meth.name, meth.key, value from "+sql_meth+" as meth left join "        
        sql2 = "(select id ,name from Measurables  where type ='type' ) as type on meth.parent = type.id "                                
#         where_class = "where type.name in ('SelectionManager') "
#         sql = sql1+sql2+where_class
        sql = sql1+sql2
        return pd.read_sql(sql,self.conn)

    def list_by_fanout(self):
        measurable_key = 'FANOUT'
        sql_meth = "(select ms.id , ms.parent,ms.name,key,value from Measurables as ms LEFT JOIN Measures as me on ms.id = me.measurable where key ='"+measurable_key+"' order by value desc)"        
        sql1 = "select meth.id as method_id, type.name as type,meth.name, meth.key, value from "+sql_meth+" as meth left join "        
        sql2 = "(select id ,name from Measurables  where type ='type' ) as type on meth.parent = type.id "                                        
        sql = sql1+sql2
        return pd.read_sql(sql,self.conn)
    
# cint0 = CustomQuery('1-tahap-0-argouml.sqlite')
# cint0.list_by_cint().to_excel("argouml-CINT-setelah-modif.xlsx")
cquery0 = CustomQuery('1-tahap-0-argouml.sqlite')
# cquery0.get_number_of_classes()
# cquery0.get_number_of_classes().to_excel("argouml-NOCS.xlsx")
cquery0.get_number_of_classes().to_csv("argouml-NOCS.csv")

In [6]:
print("branch tahap-0 i.e sesudah refactoring dilakukan ")
tahap0.get_table()

branch tahap-0 i.e sesudah refactoring dilakukan 


,level,frekuensi
0,Harm,1
1,Problem,0
2,Trouble,1
3,Warning,11
4,Bother,0


In [7]:
print("branch tahap-1 i.e sesudah refactoring dilakukan ")
tahap1.get_table()

branch tahap-1 i.e sesudah refactoring dilakukan 


NameError: name 'tahap1' is not defined

In [ ]:
tahap1.dist()
# tahap1.dist().to_excel('warning.xlsx')